In [ ]:
%matplotlib tk 
import mne
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')
from main.extraction.physionet_MI import extractPhysionet
from main.extraction.bci3_IVa import extractBCI3
from data import brain_atlas  as bm
plot = False

### Physionet

In [ ]:
# raw = extractPhysionet(runs = [3], person_id = 3)
# # raw.pick_channels(['C3','C4'])
# raw.notch_filter(60)
# raw = raw.filter(7,50,verbose = False).copy()

# event_data = mne.events_from_annotations(raw)
# event_marker, event_ids = event_data
# event_ids = dict({'T1':2, 'T2':3})
# epochs = mne.Epochs(raw, event_marker, event_ids, tmin=-2, tmax=4, preload = True)

In [ ]:
# epoch_file = '../preproc/Physionet_trial_epo.fif'
# epochs = mne.read_epochs(epoch_file)

In [ ]:
# epochs = epochs.apply_proj().copy()
# epochs = epochs.apply_baseline((-2.0,-0.2)).copy()
# epochs.equalize_event_counts()
# tmin, tmax = -0.5, 2
# epochs.crop(tmin= tmin, tmax=tmax)
# picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']
# epochs.pick_channels(picks)
# T1 = epochs['T1']
# T2 = epochs['T2']
# t1 = epochs['T1'].average()
# t2 = epochs['T2'].average()
# # mne.viz.plot_compare_evokeds([T1,-T2],picks= c);
# classes = list(event_ids.keys())

### BCI3 -IVa

In [48]:
raw = extractBCI3(runs = 0, person_id = 3)
raw = raw.filter(7,30,verbose = False).copy()

event_data = mne.events_from_annotations(raw)
event_marker, event_ids = event_data
ff= event_marker[:,-1].argsort()
event_marker = event_marker[ff,:]
event_ids = dict(right = 1, foot = 3, test = 2)
epochs = mne.Epochs(raw, event_marker, event_ids, tmin=-0.5, tmax=1, preload = True)

Used Annotations descriptions: ['-1.0', '0.0', '1.0']
Not setting metadata
Not setting metadata
280 matching events found
Setting baseline interval to [-0.5, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 280 events and 151 original time points ...
0 bad epochs dropped


/tmp/ipykernel_88047/3003246356.py:9: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw, event_marker, event_ids, tmin=-0.5, tmax=1, preload = True)


In [36]:
epoch_file = '../preproc/BCI3_ssp_car_ica_[3]_P3_epo.fif'
epochs = mne.read_epochs(epoch_file)
event_ids = epochs.event_id

Reading ../preproc/BCI3_ssp_car_ica_[3]_P3_epo.fif ...
Isotrak not found
    Read a total of 3 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 24) active
        EOG-eeg--0.200-0.200-PCA-02 (1 x 24) active
        Average EEG reference (1 x 24) active
    Found the data of interest:
        t =    -500.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
280 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated


In [37]:
epochs = epochs.apply_proj().copy()
epochs = epochs.apply_baseline().copy()
epochs.equalize_event_counts()
tmin, tmax = -0.5, 2
# epochs.crop(tmin= tmin, tmax=tmax)
picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']
epochs.pick_channels(picks)
T1 = epochs['right']
T2 = epochs['foot']
t1 = epochs['right'].average()
t2 = epochs['foot'].average()
classes = list(event_ids.keys())
# mne.viz.plot_compare_evokeds([T1,-T2],picks= c);

Projections have already been applied. Setting proj attribute to True.
Applying baseline correction (mode: mean)
Dropped 202 epochs: 0, 1, 2, 3, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256

### Epochs Analysis

In [41]:
ff = epochs.events[:,-1].argsort()

In [43]:
epochs.events

array([[ 7937,     0,     1],
       [ 8501,     0,     1],
       [ 9045,     0,     1],
       [ 9598,     0,     1],
       [10148,     0,     1],
       [12970,     0,     1],
       [13553,     0,     1],
       [14090,     0,     1],
       [14625,     0,     1],
       [15196,     0,     1],
       [15769,     0,     1],
       [17436,     0,     1],
       [18552,     0,     1],
       [21198,     0,     1],
       [21769,     0,     1],
       [22866,     0,     1],
       [23997,     0,     1],
       [25103,     0,     1],
       [25671,     0,     1],
       [26217,     0,     1],
       [26787,     0,     1],
       [27892,     0,     1],
       [28448,     0,     1],
       [30102,     0,     1],
       [30683,     0,     1],
       [31248,     0,     1],
       [31785,     0,     3],
       [32922,     0,     3],
       [33495,     0,     3],
       [34070,     0,     3],
       [37297,     0,     3],
       [37853,     0,     3],
       [38417,     0,     3],
       [38

In [45]:
ff

array([ 0, 25, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 24, 11, 12,  2,
        3,  4,  5,  1,  7,  8,  9, 10,  6, 54, 63, 55, 56, 57, 58, 59, 60,
       61, 62, 64, 70, 66, 67, 68, 69, 71, 72, 73, 74, 75, 53, 65, 52, 77,
       76, 51, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41,
       42, 43, 44, 45, 46, 47, 48, 49, 50, 38])

In [44]:
epochs.events[ff,-1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [ ]:
# epochs.plot_image();
# epochs.plot_psd();
# scale = dict(mag=1e-12, grad=4e-11, eeg=100e-6)
# epochs.plot(scalings = scale);

# t1.plot_joint(times = (0.19,2.47));

# t2.plot_joint(times = (0.19,2.47));

In [34]:
epochs_data = epochs.get_data()
# labels = epochs.events[epochs.events[:,-1]!=3]
labels = epochs.events[:,-1]
labels = labels-2
# labels = epochs.events[:,-1]-2
print(f"Data Size: {epochs_data.shape}")

# train data 
tmin, tmax = -0.5, 1
epochs_train = epochs.copy().crop(tmin= tmin, tmax=tmax)
epochs_train_data = epochs_train.get_data()
# epochs_train_data = np.transpose(epochs_train_data, axes = (1, 2, 0)) # Sklearn data = n_sam * n_freq
print(f"Train Size: {epochs_train_data.shape}")

Data Size: (78, 7, 151)
Train Size: (78, 7, 151)


In [ ]:
if plot == True:
    clas = 'right'
    # epochs['right'].plot_image(combine = 'mean', title = 'right');
    epochs[clas].average().plot_image(show_names = 'all', titles = clas, picks = picks);
    epochs[clas].plot_topo_image(title = clas);
    epochs[clas].plot_image(title = clas, combine = 'mean');
    epochs[clas].plot_psd();
    epochs[clas].average().plot_joint(picks=picks);

### Power

In [ ]:
from mne.time_frequency import tfr_morlet

In [ ]:
frequencies =  np.arange(5,30,1) #np.logspace(*np.log10([5, 30]), num=25)
chpicks = [1] #baseline = (-0.5, 0.),

In [ ]:
power = tfr_morlet(epochs[classes[0]], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
if plot == True:
    for pick in chpicks:
        # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
        # power.plot_joint(timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = classes[0])
        power.plot_joint(mode = 'mean', title = classes[0])

In [ ]:
power = tfr_morlet(epochs[classes[1]], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
if plot == True:
    for pick in chpicks:
        # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
        power.plot_joint(timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = classes[1])

In [ ]:
# 1/f removal- Power Normalization - Average TFR
clas = 'foot'
baset = [epochs.baseline[1] , -0.2]
power = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
powerDB = np.zeros(power.data.shape)
t = np.arange(np.abs(baset[1]-baset[0])*epochs.info['sfreq'], dtype = np.int64)
for ch in range(power.data.shape[0]):
    for f in range(power.data.shape[1]):
        baseline = np.mean(power.data[ch,f,t])
        activity = power.data[ch,f,:]
        powerDB[ch,f,:] = 10*np.log10(activity/baseline)

newPower = mne.time_frequency.AverageTFR(epochs.info, powerDB,epochs.times, frequencies,1).copy()
if plot == True:
    newPower.plot_joint(title = clas, mode = 'mean');

In [52]:
len((epochs.events[:,-1]).unique())
# ff

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [ ]:
# 1/f removal- Power Normalization - Epochs TFR
def powerEPdBcalc(clas):
    baset = [epochs.baseline[1] , -0.2]
    power = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False, average=False)
    shp = power.data.shape
    powerdB = np.zeros((shp[0],shp[2],shp[3]))
    powerEp = powerdB.copy()
    t = np.arange(np.abs(baset[1]-baset[0])*epochs.info['sfreq'], dtype = np.int64)
    for ep in range(power.data.shape[0]):
        EPpower  = np.mean(power.data[ep,:,:,:],axis=1)
        for f in range(EPpower.shape[0]):
            baseline = np.mean(EPpower[f,t])
            activity = EPpower[f,:]
            powerdB[ep,f,:] = 10*np.log10(activity/baseline)
            powerEp[ep,f,:] = activity
    return powerdB, powerEp

# newEPPower = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False, average=False)
powerEpdB, powerEp = {}, {}
powerEpdB[classes[0]], powerEp[classes[0]]= powerEPdBcalc(classes[0])
powerEpdB[classes[1]], powerEp[classes[1]]= powerEPdBcalc(classes[1])

# newPower.plot_joint(title = clas, mode = 'mean');

In [ ]:
powerEpdB

In [ ]:
train_data = np.vstack((powerEp[classes[0]],powerEp[classes[1]]))
# train_data = train_data.reshape(train_data.shape[0],-1)
labels = np.concatenate([-np.ones(powerEpdB[classes[0]].shape[0]),
                     np.ones(powerEpdB[classes[1]].shape[0])])
print(train_data.shape)
print(labels.shape)

In [ ]:
if plot == True:
    scaler = MinMaxScaler((0,1))
    data = scaler.fit_transform(powerEp[classes[0]][10,:,:])
    plt.figure()
    plt.imshow(data)
    plt.show()

### Classification

#### ML

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
clf = LDA()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
scaler = MinMaxScaler((0,1))
train_data = scaler.fit_transform(train_data)

In [ ]:
# pipe = Pipeline([('scatter', scat_obj), ('clf', clf)])
# scores = cross_val_score(pipe, epochs_train_data, labels, cv = cv, verbose=False)

In [ ]:
sfreq = epochs.info['sfreq']
w_length = int(sfreq * 0.05)   # running classifier: window length
w_step = int(sfreq * 0.01)  # running classifier: window step size
w_start = np.arange(0, train_data.shape[1] - w_length, w_step)

scores_windows = []

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = train_data[train_idx]
    X_test = train_data[test_idx]

    # fit classifier
    clf.fit(X_train, y_train)

    # running classifier: test classifier on sliding window
    score_this_window = []
    # for n in w_start:
    #     X_test = (train_data[test_idx]) #[:, n:(n + w_length)])
    scores_windows.append(clf.score(X_test, y_test))
    # scores_windows.append(score_this_window)
w_times = (w_start + w_length / 2.) / sfreq + epochs.tmin
np.mean(scores_windows)
# plt.scatter(scores_windows)

#### DL

In [ ]:
import torch, torchvision
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary

alexnet = models.alexnet(pretrained=True, progress=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
alexnet.eval();
alexnet

In [ ]:
# Freesze model parameters
for param in alexnet.parameters():
    param.requires_grad = False

# change final layer of alexnet model
alexnet.classifier[6] = nn.Linear(4096,2)
alexnet.classifier.add_module('7',nn.LogSoftmax(dim=1))
alexnet

In [ ]:
summary(alexnet,(3,224,224))

In [ ]:
loss = nn.NLLLoss()
optimizer = optim.Adam(alexnet.parameters())
optimizer

In [ ]:
from PIL import Image
scaler = MinMaxScaler((0,1))
data = scaler.fit_transform(powerEp[classes[0]][10,:,:])
input_image = Image.fromarray(np.uint8(cm.gist_earth(data)*255)[:,:,:3])
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

In [ ]:
plt.imshow(powerEpdB[classes[0]][0])
plt.show()
plt.figure()
plt.imshow(powerEp[classes[0]][0])
plt.show()

In [ ]:
np.uint8(cm.gist_earth(data)*255).shape

In [ ]:
plt.figure()
plt.imshow(im,  interpolation='nearest', cmap='gray')
plt.show()